In [1]:
# 1) Install deps
!pip install -q diffusers==0.30.0 transformers==4.44.2 accelerate==0.33.0 safetensors pillow tqdm pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 80.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.

In [2]:
# 2) Imports and strong silencing
import os
import sys
import logging
from pathlib import Path
import zipfile
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch

# Environment-level silencing BEFORE importing diffusers/transformers
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"   # HF hub
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"
os.environ["BITSANDBYTES_NOWELCOME"] = "1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["DIFFUSERS_VERBOSITY"] = "error"        # diffusers internal
os.environ["ACCELERATE_DISABLE_RICH"] = "1"        # prevent rich progress
os.environ["ACCELERATE_LOG_LEVEL"] = "ERROR"
os.environ["TQDM_DISABLE"] = "1"                   # disable any nested tqdm that libs might spawn

# Silence python loggers
for name in [
    "diffusers",
    "transformers",
    "accelerate",
    "urllib3",
    "filelock",
    "torch",
    "huggingface_hub",
]:
    logging.getLogger(name).setLevel(logging.ERROR)
logging.getLogger().setLevel(logging.ERROR)

# 3) Import pipelines after env is set
from diffusers import AutoPipelineForText2Image
from transformers.utils import logging as hf_logging
hf_logging.set_verbosity_error()

# Helper: context manager to suppress stdout/stderr temporarily (for model calls)
import contextlib
@contextlib.contextmanager
def suppress_stdout_stderr():
    with open(os.devnull, 'w') as devnull:
        old_stdout, old_stderr = sys.stdout, sys.stderr
        try:
            sys.stdout, sys.stderr = devnull, devnull
            yield
        finally:
            sys.stdout, sys.stderr = old_stdout, old_stderr

# 4) Config
CSV_PATH = "/kaggle/input/newsimagedataset-v2/newsarticles.csv"   # change if needed
GROUP_NAME = "CodingSoft"
APPROACH_NAME = "SDTURBO"
TASK_TYPE = "GEN"
OUTPUT_BASE = Path("/kaggle/working/output_images")
HEIGHT, WIDTH = 260, 460

OUTPUT_LARGE = OUTPUT_BASE / f"{TASK_TYPE}_{APPROACH_NAME}_LARGE"
OUTPUT_SMALL = OUTPUT_BASE / f"{TASK_TYPE}_{APPROACH_NAME}_SMALL"
OUTPUT_LARGE.mkdir(parents=True, exist_ok=True)
OUTPUT_SMALL.mkdir(parents=True, exist_ok=True)

# 5) Load model quietly
device = "cuda" if torch.cuda.is_available() else "cpu"
with suppress_stdout_stderr():
    pipe = AutoPipelineForText2Image.from_pretrained(
        "stabilityai/sd-turbo",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    ).to(device)

# Disable any progress callbacks and safety prints
# Many pipelines accept guidance on progress via callbacks, we won't use them.
# We’ll also avoid enable_model_cpu_offload logs & similar.
# If needed: pipe.set_progress_bar_config(disable=True) — diffusers supports it in some versions.
try:
    pipe.set_progress_bar_config(disable=True)
except Exception:
    pass

# 6) Prepare data
df = pd.read_csv(CSV_PATH)

style_suffix = "in style of digital illustration, high detail, non photo-realistic"
df["prompt"] = df["article_title"].fillna('') + " " + df["article_tags"].fillna('') + ", " + style_suffix

# Load subset IDs for SMALL folder
SMALL_CSV_PATH = "/kaggle/input/newsimagedataset-v2/subset.csv"  # change if needed
small_df = pd.read_csv(SMALL_CSV_PATH, header=None, usecols=[0])
small_ids = set(small_df[0].astype(str))

df["article_id"] = df["article_id"].astype(str)

# 7) One global progress bar, no internal bars
total = len(df)
gen_height, gen_width = 264, 464  # divisible by 8, then resize to 460x260

# Optional memory tweaks
# pipe.enable_attention_slicing()
# pipe.enable_vae_slicing()

# We also wrap the per-image call in suppress_stdout_stderr() to kill any residual prints/bars
with tqdm(total=total, desc="Generating images", leave=True) as pbar:
    for idx in range(total):
        row = df.iloc[idx]
        article_id = row["article_id"]
        prompt = row["prompt"]

        with suppress_stdout_stderr():
            out = pipe(
                prompt,
                height=gen_height,
                width=gen_width,
                guidance_scale=0.0,
                num_inference_steps=2  # keep same behavior
            )
        image = out.images[0]

        image = image.resize((WIDTH, HEIGHT), Image.LANCZOS)

        filename = f"{article_id}_{GROUP_NAME}_{APPROACH_NAME}.png"
        image.save(OUTPUT_LARGE / filename)

        if article_id in small_ids:
            image.save(OUTPUT_SMALL / filename)

        pbar.update(1)

# 8) Create required submission ZIP
ZIP_PATH = Path(f"/kaggle/working/{GROUP_NAME}.zip")
with zipfile.ZipFile(ZIP_PATH, 'w') as zipf:
    for img_file in OUTPUT_LARGE.iterdir():
        zipf.write(img_file, arcname=f"{TASK_TYPE}_{APPROACH_NAME}_LARGE/{img_file.name}")
    for img_file in OUTPUT_SMALL.iterdir():
        zipf.write(img_file, arcname=f"{TASK_TYPE}_{APPROACH_NAME}_SMALL/{img_file.name}")

print(f"✅ Submission ZIP ready: {ZIP_PATH}")

# 9) Optional: full working archive like your original
import shutil
parent_folder = "/kaggle/working/"
zip_path = "/kaggle/working/CodingSoft"
shutil.make_archive(zip_path, "zip", parent_folder)
print(f"📦 Full working archive: {zip_path}.zip")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
/usr/local/lib/python3.11/dist-packages/accelerate/utils/imports.py:282: UserWarning: `ACCELERATE_DISABLE_RICH` is deprecated and will be removed in v0.22.0 and deactivated by default. Please use `ACCELERATE_ENABLE_RICH` if you wish to use `rich`.
  warnings.warn(
E0000 00:00:1754916313.532820      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754916313.599622      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Generating images: 100%|██████████| 8500/8500 [42:24<00:00,  3.34it/s]


✅ Submission ZIP ready: /kaggle/working/CodingSoft.zip
📦 Full working archive: /kaggle/working/CodingSoft.zip
